Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, re, subprocess, sys, json, glob
#from molmod import *
#from molmod.io import FCHKFile
#from molmod.io.xyz import XYZReader, XYZFile
import requests
#import wts

In [2]:
"abc"

'abc'

print(2)

get_dihed.py

In [3]:
os.getcwd()

'/home/hagen/anaconda3/bin/hemejpn'

take_properties.py

In [4]:
df = pd.DataFrame({2:["a","b"],3:[1,2]})
df  =df.set_index(2)
#df.index = df.index.str.upper()

df

,3
2,
a,1
b,2


In [5]:
if(False):
    wts.TCL-Skript()
    wts.dihedpdb()        
    wts.prepare_gaussian_logs().core()
    wts.onecsv()

    wts.read_redpot_lit()

In [6]:
dfex_1 =pd.read_csv("tables/mono.csv")
dfex_1  =dfex_1.set_index("pdb")
dfex_1 = dfex_1[[ "Elektrode","Ligand","EMv"]]
dfex_3 = pd.read_csv("tables/duplicated_moded.csv")
dfex_4 = dfex_3.drop_duplicates("PDB", keep=False)
dfex_4= dfex_4.set_index("pdb")#.rename({"PDB":"pdb"})
dfex_4 = dfex_4[[ "Elektrode","Ligand","EMv"]]

dfex_lit = dfex_1.append(dfex_4)
dfex_lit.index = dfex_lit.index.str.upper()
dfex_lit =dfex_lit.rename(columns = {"EMv":"Em_lit"})
df_axlig = pd.read_csv("tables/axligand.csv")
df_axlig = df_axlig.rename(columns={"# PDB": "pdb","ligand" : "ligand_pydish"}).set_index("pdb")
df_axlig = df_axlig[["ligand_pydish","saddling","ruffling"]]

df_calc = pd.read_csv("tables/calculated.csv")
df_calc = df_calc.drop("Unnamed: 0", axis=1).set_index("pdb")
_dihedral = pd.read_csv("tables/Dihedral.csv")
_dihedral = _dihedral.rename(columns={"Unnamed: 0":"PDB"})
_dihedral = _dihedral.set_index("PDB")

df_ruffling = pd.read_csv("tables/Ruffling.csv").rename(columns = {"Unnamed: 0":"PDB"}).set_index("PDB")
df_saddling = pd.read_csv("tables/Saddling.csv").rename(columns = {"Unnamed: 0":"PDB"}).set_index("PDB")

dfHemetype = pd.read_csv("tables/Hemetypes.csv")
dfHemetype = dfHemetype.set_index("pdb")


pdb_calc  = list(set(df_calc.index))
pdb_lit  = list(set(dfex_lit.index))
#print(    ([i for i in pdb_calc  if not i in pdb_lit])   )

pdbavaible = [i for i in pdb_calc if i in pdb_lit]
pdbavaiblepdb_calc  = list(set(df_calc.index))
pdb_lit  = list(set(dfex_lit.index))
#print(    ([i for i in pdb_calc  if not i in pdb_lit])   )

pdbavaible = [i for i in pdb_calc if i in pdb_lit]
#pdbavaible

df_ruffling = df_ruffling.loc[pdbavaible]
df_saddling = df_saddling.loc[pdbavaible]

In [7]:
if not all(i in df_ruffling.index for i in pdbavaible):
    print(f"ruffling fehlt{[i for i in pdbavaible if i not in df_ruffling.index]}")
    
if not all(i in _dihedral.index for i in pdbavaible):
    print(f"Dihedral fehlt{[i for i in pdbavaible if i not in _dihedral.index]}")

if not all(i in df_saddling.index for i in pdbavaible):
    print(f"saddling fehlt{[i for i in pdbavaible if i not in df_saddling.index]}")    

In [8]:
df_ruffling.head()

,C3_SW_C1_SW_C1_WS_C3_WS,C3_SO_C1_SO_C1_OS_C3_OS,C3_WN_C1_WN_C1_NW_C3_NW,C3_NO_C1_NO_C1_ON_C3_ON
PDB,,,,
1JP6,1.566411,-0.303655,0.418476,-0.857936
1CTJ,-4.498742,2.112754,8.096088,1.480299
1ES1,-2.213717,-7.884052,12.262675,14.270593
1JP9,0.755836,-0.199399,1.356857,-1.431851
1A6G,7.471468,-1.022543,-2.678883,-9.072572


In [9]:
df_saddling.head()

,C1_SW_N_S_N_N_C1_NW,C1_SO_N_S_N_N_C1_NO,C1_WS_NW_NO_C1_OS,C1_WN_NW_NO_C1_ON
PDB,,,,
1JP6,0.423901,-0.142268,179.409164,179.520162
1CTJ,15.501774,5.853905,-176.582978,-176.343689
1ES1,-7.512205,-7.130937,-0.819562,-8.460821
1JP9,0.593972,-0.172962,179.413463,179.513896
1A6G,11.105041,-8.137583,-179.579804,179.832818


In [10]:
if(False):    
    df_pydish = pd.read_csv("tables/pydish.csv")
    df_pydish = df_pydish.rename(columns = { "# PDB":"PDB"})
    df_pydish["PDB"] = df_pydish["PDB"].str.upper()
    df_pydish = df_pydish.set_index("PDB")

    df_pydish["HemeA"] = ["_A_" in HEM_ for HEM_ in df_pydish["Heme"]]
    df_pydish = df_pydish[df_pydish["HemeA"]]
    df_pydish = df_pydish.loc[pdbavaible][["ligand","saddling","ruffling","doming"]]
    df_pydish[df_pydish.index.duplicated(keep =False)]
    df_pydish = df_pydish[[a or b for a,b in zip(df_pydish.index!="2GSM", list(df_pydish["ligand"]!="HIS-HOH"))]]
    df_pydish = df_pydish.loc[pdbavaible]


In [11]:
def blindserie(columns,spin,met,pdb):
    ser={}
 #   spin= 5
  #  met = "nbo"
   # pdb = "1CCC"
    for c in columns:
        if c ==  "spin":
            ser[c] = [spin]
        elif c ==  "Ox":
            ser[c] = {1:[0],2:[1],5:[0],6:[1]}[spin]
        elif c == "method":
            ser[c] = [met]
        else:
            ser[c] = [None]
    return pd.DataFrame(ser,index=[pdb])

In [12]:
ls = {}
for spin in [1,2,5,6]:
    for met in ["nbo","chloro"]:
        df_oxmet = df_calc[df_calc["spin"] == spin]
        df_oxmet = df_oxmet[df_oxmet["method"] == met]
        #print(spin, met, [a1 for a1 in pdbavaible if a1 not in df_oxmet.index])
        for pdb in  [a1 for a1 in pdbavaible if a1 not in df_oxmet.index]:
            df_oxmet = df_oxmet.append(blindserie(df_oxmet.columns,spin,met,pdb))    
        df_oxmet = df_oxmet[[n for n in df_oxmet.columns if n not in ["spin", "method", "Ox"]]]

        df_oxmet["diff"] = df_oxmet["lumo"] - df_oxmet["homo"]

        for c in df_oxmet.columns:
            df_oxmet = df_oxmet.rename(columns = {c:c+"_"+str(spin)+"_"+met}) 
        df_oxmet = df_oxmet.loc[pdbavaible]                                    
        ls[str(spin)+met] = df_oxmet


In [13]:
for k in ls.keys():
    print(k,ls[k].shape)

1nbo (33, 13)
1chloro (33, 13)
2nbo (33, 13)
2chloro (33, 13)
5nbo (33, 13)
5chloro (33, 13)
6nbo (33, 13)
6chloro (33, 13)


In [14]:
dfex_lit.loc[pdbavaible]
df_work = dfex_lit.loc[pdbavaible].copy()
for k in ls.keys():
    df_work = pd.concat([df_work, ls[k] ],axis=1)
#df_work = pd.concat([df_work, _dihedral.loc[pdbavaible] ],axis=1)
#df_work = pd.concat([df_work,df_pydish ],axis=1)

In [15]:
#df_work =df_work.loc[[i for i in df_work.index if i != "1AW3"]]

In [16]:
df_work["Hemetype"]=[dfHemetype.loc[i][0] for i in df_work.index]
df_work2 = df_work[[c for c in df_work.columns if True not in df_work[c].isna().value_counts().index]]

In [17]:
df_work.head()

,Elektrode,Ligand,Em_lit,e_1_nbo,edisp_1_nbo,homo_1_nbo,lumo_1_nbo,chem_pot_1_nbo,dipole_1_nbo,qpole1_1_nbo,...,chem_pot_6_chloro,dipole_6_chloro,qpole1_6_chloro,qpole2_6_chloro,qpole3_6_chloro,qpole4_6_chloro,polar-iso_6_chloro,polar-aniso_6_chloro,diff_6_chloro,Hemetype
pdb,,,,,,,,,,,,,,,,,,,,,
1JP6,NHE,mono-His,50.0,-3438.493534,-79.663631,-0.15456,-0.09723,-0.12590,3.6011,17.916237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heme B
1CTJ,SHE,NaN,358.0,-3881.897804,-95.266387,-0.17997,-0.06395,-0.12196,5.5992,23.709088,...,-0.16224,9.1308,34.669962,-5.950271,-28.719691,45.411812,NaN,NaN,0.11522,Heme B
1ES1,SHE,Bis-His,12.0,-3627.495832,-90.676104,-0.17333,-0.06587,-0.11960,2.1560,36.474156,...,-0.16263,2.5347,38.140716,17.519271,-55.659987,69.711356,NaN,NaN,0.10906,Heme B
1JP9,NHE,mono-His,50.0,-3438.491798,-80.459073,-0.15478,-0.09966,-0.12722,6.3832,18.809174,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Heme B
1A6G,NHE,mono-His,50.0,-3475.471726,-81.291268,-0.16762,-0.09782,-0.13272,3.8308,28.314378,...,-0.16626,2.4694,41.033578,-16.921591,-24.111986,50.512203,NaN,NaN,0.07653,Heme B


In [18]:
df_work2.head()

,Elektrode,Em_lit,e_1_nbo,edisp_1_nbo,homo_1_nbo,lumo_1_nbo,chem_pot_1_nbo,dipole_1_nbo,qpole1_1_nbo,qpole2_1_nbo,...,homo_5_nbo,lumo_5_nbo,chem_pot_5_nbo,dipole_5_nbo,qpole1_5_nbo,qpole2_5_nbo,qpole3_5_nbo,qpole4_5_nbo,diff_5_nbo,Hemetype
pdb,,,,,,,,,,,,,,,,,,,,,
1JP6,NHE,50.0,-3438.493534,-79.663631,-0.15456,-0.09723,-0.12590,3.6011,17.916237,-3.881981,...,-0.17029,-0.07184,-0.12107,3.5970,16.252285,-2.943805,-13.308481,21.211280,0.09845,Heme B
1CTJ,SHE,358.0,-3881.897804,-95.266387,-0.17997,-0.06395,-0.12196,5.5992,23.709088,4.996309,...,-0.16828,-0.06921,-0.11875,5.6534,24.033301,3.824088,-27.857389,36.989962,0.09907,Heme B
1ES1,SHE,12.0,-3627.495832,-90.676104,-0.17333,-0.06587,-0.11960,2.1560,36.474156,12.834596,...,-0.10977,-0.06186,-0.08582,3.0407,41.162319,7.112970,-48.275288,63.839128,0.04791,Heme B
1JP9,NHE,50.0,-3438.491798,-80.459073,-0.15478,-0.09966,-0.12722,6.3832,18.809174,-1.325091,...,-0.17150,-0.07304,-0.12227,5.8843,22.443142,-8.049803,-14.393339,27.850712,0.09846,Heme B
1A6G,NHE,50.0,-3475.471726,-81.291268,-0.16762,-0.09782,-0.13272,3.8308,28.314378,-9.751173,...,-0.15907,-0.07591,-0.11749,2.4858,29.722478,-7.264996,-22.457482,37.954504,0.08316,Heme B


In [19]:
df_work_distortion = df_work2.copy()
df_work_distortion["ruffling_mean"] = df_ruffling.mean(axis = 1)
df_work_distortion["ruffling_max"] = df_ruffling.max(axis = 1)
df_work_distortion["saddling_mean"] = df_saddling.mean(axis = 1)
df_work_distortion["saddling_max"] = df_saddling.max(axis = 1)
df_work_distortion_only = df_work_distortion[["Em_lit","ruffling_mean","ruffling_max","saddling_mean","saddling_max"]]


In [20]:
df_work_HemeB = df_work2[df_work2["Hemetype"] == "Heme B"]
df_work_distortion_HemeB = df_work_distortion[df_work_distortion["Hemetype"] == "Heme B"]
df_work_distortion_only_HemeB =  df_work_distortion_only[df_work2["Hemetype"] == "Heme B"]

In [21]:
df_work2.head()

,Elektrode,Em_lit,e_1_nbo,edisp_1_nbo,homo_1_nbo,lumo_1_nbo,chem_pot_1_nbo,dipole_1_nbo,qpole1_1_nbo,qpole2_1_nbo,...,homo_5_nbo,lumo_5_nbo,chem_pot_5_nbo,dipole_5_nbo,qpole1_5_nbo,qpole2_5_nbo,qpole3_5_nbo,qpole4_5_nbo,diff_5_nbo,Hemetype
pdb,,,,,,,,,,,,,,,,,,,,,
1JP6,NHE,50.0,-3438.493534,-79.663631,-0.15456,-0.09723,-0.12590,3.6011,17.916237,-3.881981,...,-0.17029,-0.07184,-0.12107,3.5970,16.252285,-2.943805,-13.308481,21.211280,0.09845,Heme B
1CTJ,SHE,358.0,-3881.897804,-95.266387,-0.17997,-0.06395,-0.12196,5.5992,23.709088,4.996309,...,-0.16828,-0.06921,-0.11875,5.6534,24.033301,3.824088,-27.857389,36.989962,0.09907,Heme B
1ES1,SHE,12.0,-3627.495832,-90.676104,-0.17333,-0.06587,-0.11960,2.1560,36.474156,12.834596,...,-0.10977,-0.06186,-0.08582,3.0407,41.162319,7.112970,-48.275288,63.839128,0.04791,Heme B
1JP9,NHE,50.0,-3438.491798,-80.459073,-0.15478,-0.09966,-0.12722,6.3832,18.809174,-1.325091,...,-0.17150,-0.07304,-0.12227,5.8843,22.443142,-8.049803,-14.393339,27.850712,0.09846,Heme B
1A6G,NHE,50.0,-3475.471726,-81.291268,-0.16762,-0.09782,-0.13272,3.8308,28.314378,-9.751173,...,-0.15907,-0.07591,-0.11749,2.4858,29.722478,-7.264996,-22.457482,37.954504,0.08316,Heme B


In [22]:
"_________________________________________________________________________________________________________________________________________________________________________________________"

'_________________________________________________________________________________________________________________________________________________________________________________________'

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import numpy as np

In [24]:
PCA()

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [25]:
def pca_show(df_work2, **kwargs):
    default_value_order=[1,2,0]
    order = kwargs.pop('order', default_value_order)    
    
    default_value_n_components=6
    num_of_componentes = kwargs.pop('num_comp', default_value_n_components) 
    if num_of_componentes>df_work2.shape[1]:
        num_of_componentes=df_work2.shape[1]
        
    
    
    best_n=3
    best_par = kwargs.pop('best_par', default_value_n_components)     
    if best_par> num_of_componentes:
        best_par== num_of_componentes
        
    default_value_whiten = False
    whiten_ = kwargs.pop('whiten', default_value_whiten)
    
    hk1,hk2,hk3=order   #x,y,c
    df_work2 = df_work2[[n for n in df_work2.columns if not n in [i for i in df_work2.dtypes[df_work2.dtypes==object].index]]]
    X = df_work2.values
    s = StandardScaler()
    X = s.fit_transform(X)
    p = PCA(n_components = num_of_componentes, whiten = whiten_)
    p.fit(X)
    X_transformed = p.transform(X)
    X_transformed_filtered = X_transformed
    #print(p.explained_variance_ratio_)
    #print("\n Sum:",sum(p.explained_variance_ratio_))
    print("noise variance: ",p.noise_variance_)
    print( "number of componentes: ", num_of_componentes) 
    #print(f"\n the best parameter of the first {best_par} components")
   # for i in range(best_par):
    #    print(p.components_[i].max())
    if(False):
        print(p.components_[0].max())
        print(p.components_[1].max())
        print(p.components_[2].max())
        plt.scatter(X_transformed_filtered[:, hk1], X_transformed_filtered[:, hk2] , c = X_transformed_filtered[:, hk3] )

        plt.xlabel(str(hk1))
        plt.xlabel(f"HK{hk1+1}")
        plt.ylabel(str(hk2))
        plt.ylabel(f"HK{hk2+1}")

        plt.title(f"Color = HK{hk3+1}")
        #plt.show()
   # p,cols  =pca_show(df_work2,num_comp=20)
    #return pd.DataFrame(p.components_,columns = cols)
    cols = list(df_work2.columns)
    #print("noise_variance_: ",p.noise_variance_)
    return pd.DataFrame(p.components_,columns = cols)

In [26]:
def best_components(df, **kwargs):
        
    default_value_n_components=6
    num_of_componentes = kwargs.pop('num_comp', default_value_n_components) 
    
    default_value_min_Em_lit = 0.1
    min_Em_lit = kwargs.pop('min_Em_lit', default_value_min_Em_lit)

    default_value_min_PC = 0.15
    min_PC = kwargs.pop('min_PC', default_value_min_PC)
    
    
    
    df  =pca_show(df,num_comp=num_of_componentes, whiten = True) #,num_comp=32)
    df_abs = df.abs()
 #   emv = list(df_abs["Em_lit"])
  #  emv.sort(reverse=True)
   # for i,v in enumerate(emv[:5]):
    #    print(v,"  ",sum(emv[:i+1]))
    bo = df[df_abs["Em_lit"]> min_Em_lit ].index

    list_of_df = []

    for b in bo:

        numname = [(np.round(v,4),i) for i,v in zip(df.loc[b].index,df.loc[b]) if abs(v)>min_PC]
        names = [i[1] for i in numname] 
        nums = [i[0] for i in numname] 
        list_of_df.append(pd.DataFrame({df.loc[b]["Em_lit"]:nums}, index= names))

    if len(list_of_df)>1:
        a = []
        for i in list_of_df:
            a = a+ list( i.index)
        a = list(set(a))


        list_of_best = [i for i in a if all([i in j.index for j in list_of_df]) ]
        Matrix = [[ld.loc[[lb]].iloc[0,0] for ld in list_of_df] for lb in list_of_best]
        #print(pd.DataFrame(Matrix, index = list_of_best, columns = [df.loc[b]["Em_lit"] for b in bo]) )
        df_bo = pd.DataFrame(Matrix, index = list_of_best, columns = [(df.loc[b]["Em_lit"]) for b in bo])
        df_bo["mean"] = df_bo.mean(axis = 1)
        list_of_df.append( df_bo)
        return list_of_df

In [33]:
df_work2

,Elektrode,Em_lit,e_1_nbo,edisp_1_nbo,homo_1_nbo,lumo_1_nbo,chem_pot_1_nbo,dipole_1_nbo,qpole1_1_nbo,qpole2_1_nbo,...,homo_5_nbo,lumo_5_nbo,chem_pot_5_nbo,dipole_5_nbo,qpole1_5_nbo,qpole2_5_nbo,qpole3_5_nbo,qpole4_5_nbo,diff_5_nbo,Hemetype
pdb,,,,,,,,,,,,,,,,,,,,,
1JP6,NHE,50.0,-3438.493534,-79.663631,-0.15456,-0.09723,-0.12590,3.6011,17.916237,-3.881981,...,-0.17029,-0.07184,-0.12107,3.5970,16.252285,-2.943805,-13.308481,21.211280,0.09845,Heme B
1CTJ,SHE,358.0,-3881.897804,-95.266387,-0.17997,-0.06395,-0.12196,5.5992,23.709088,4.996309,...,-0.16828,-0.06921,-0.11875,5.6534,24.033301,3.824088,-27.857389,36.989962,0.09907,Heme B
1ES1,SHE,12.0,-3627.495832,-90.676104,-0.17333,-0.06587,-0.11960,2.1560,36.474156,12.834596,...,-0.10977,-0.06186,-0.08582,3.0407,41.162319,7.112970,-48.275288,63.839128,0.04791,Heme B
1JP9,NHE,50.0,-3438.491798,-80.459073,-0.15478,-0.09966,-0.12722,6.3832,18.809174,-1.325091,...,-0.17150,-0.07304,-0.12227,5.8843,22.443142,-8.049803,-14.393339,27.850712,0.09846,Heme B
1A6G,NHE,50.0,-3475.471726,-81.291268,-0.16762,-0.09782,-0.13272,3.8308,28.314378,-9.751173,...,-0.15907,-0.07591,-0.11749,2.4858,29.722478,-7.264996,-22.457482,37.954504,0.08316,Heme B
1CCG,SHE,-127.0,-3399.152268,-76.555113,-0.18419,-0.08090,-0.13254,6.2218,30.355839,3.692074,...,-0.17756,-0.08209,-0.12982,6.4688,30.387370,3.675496,-34.062866,45.794982,0.09547,Heme B
1A6M,NHE,50.0,-3512.449877,-80.498063,-0.16727,-0.11119,-0.13923,5.1865,28.486568,-10.967490,...,-0.17179,-0.07832,-0.12505,4.8283,31.510732,-11.630249,-19.880483,39.031046,0.09347,Heme B
1F1F,SHE,314.0,-3881.438683,-95.645001,-0.17910,-0.07173,-0.12541,9.1415,24.824175,-2.292980,...,-0.17352,-0.07497,-0.12424,9.2322,24.456697,-1.840017,-22.616681,33.362103,0.09855,Heme C
1FHF,SHE,-443.0,-3455.422494,-78.685821,-0.16090,-0.09964,-0.13027,4.3885,42.098369,9.593717,...,-0.17172,-0.07166,-0.12169,5.3105,37.296808,7.870546,-45.167353,59.102345,0.10006,Heme B


## dwork2

In [35]:
best_components(df_work2)[-1]

noise variance:  0.1946707803789098
number of componentes:  6


,-0.18286386299211416,-0.2355655939010977,mean
homo_5_nbo,-0.1909,-0.2290,-0.20995
chem_pot_1_nbo,-0.2511,0.1728,-0.03915
chem_pot_2_nbo,-0.2074,0.2537,0.02315
lumo_2_nbo,-0.1956,0.1815,-0.00705
Em_lit,-0.1829,-0.2356,-0.20925


## HemeB


In [28]:
best_components(df_work_HemeB, min_Em_lit=0.1,min_PC=0.1)[-1]

noise variance:  0.2360823735040667
number of componentes:  6


,0.18976335184464332,0.10385320890250228,-0.35834370332691795,mean
diff_1_nbo,0.2072,-0.1187,-0.2191,-0.043533
chem_pot_1_nbo,0.2272,0.1244,0.1115,0.154367
Em_lit,0.1898,0.1039,-0.3583,-0.021533


## Distortion

In [29]:
best_components(df_work_distortion, min_Em_lit=0.1,min_PC=0.15)[-1]

noise variance:  0.27587875412908897
number of componentes:  6


,-0.18748286166700337,-0.12375908959377582,-0.18411269809391087,mean
ruffling_max,0.1640,0.1617,0.1557,0.160467
homo_5_nbo,-0.1816,0.2925,-0.2535,-0.047533


## Distortion HemeB

In [30]:
best_components(df_work_distortion_HemeB, min_Em_lit=0.1,min_PC=0.15)[-1]

noise variance:  0.3300079621256515
number of componentes:  6


,0.18830056627096425,0.11533237762538696,-0.3692271375244068,mean
diff_1_nbo,0.2030,-0.2683,-0.2606,-0.108633
diff_2_nbo,0.2095,-0.1565,-0.2297,-0.058900


## Distortion only

In [31]:
best_components(df_work_distortion_only, min_Em_lit=0.1,min_PC=0.15)[-1]

noise variance:  0.0
number of componentes:  5


,0.34263842117342813,-0.660068944510137,0.16706714717028195,-0.6099983900921677,-0.21656046095813652,mean
Em_lit,0.3426,-0.6601,0.1671,-0.61,-0.2166,-0.1954


## Distortion only Heme B

In [32]:
best_components(df_work_distortion_only_HemeB, min_Em_lit=0.1,min_PC=0.15)[-1]

noise variance:  0.0
number of componentes:  5


AttributeError: type object 'object' has no attribute 'dtype'

In [ ]:
plt.scatter(df_work_distortion_only_HemeB["ruffling_max"], df_work_distortion_only_HemeB["saddling_max"], c = list(df_work_distortion_only_HemeB["Em_lit"]))

In [ ]:
plt.scatter(df_work_distortion_only_HemeB["ruffling_max"] , list(df_work_distortion_only_HemeB["Em_lit"]))

In [ ]:
df_work_distortion_only_HemeB

In [ ]:
[ld.loc[[lb]].iloc[0,0] for ld in list_of_df] 

In [ ]:
ld.loc[[lb]]

def funktion(dataframe): 
    return datframe
        0.22
    emv 0.22
    homo 0.5
    lumo 0.4

In [ ]:
[[ld.loc[lb] for ld in list_of_df] for lb in list_of_best]

In [ ]:
[i for i in a if all([i in j.index for j in list_of_df]) ]

In [ ]:
i = "chem_pot_2_nbo" 

In [ ]:
for i in df_10pc.columns:
    if df_10pc[i].loc["sum"]>0.1:
        print(df_10pc[i].loc["sum"], i)

In [ ]:
df_work3 = df_work2[[i  for i in df_10pc.columns if df_10pc[i].loc["sum"]>0.1]]

In [ ]:
df  =pca_show(df_work3,num_comp=3)

In [ ]:
df

In [ ]:
pd.concat()

In [ ]:
dicti= {}
for a,b in zip( df_10pc.columns,[df_10pc[col].sum(axis=0) for col in df_10pc.columns]):
    dicti[a] =b
df_10pc = pd.concat([df_10pc,pd.DataFrame(dicti,index=["sum"])    ])

In [ ]:
df_10pc

In [ ]:
df  =pca_show(df_work2[["Em_lit",]],num_comp=32)

In [ ]:
for c in df_work2:
    if True  in df_work[c].isna().value_counts().index:
        print(c)

In [ ]:
df_work2.dtypes[df_work2.dtypes==object]

In [ ]:
df_work2

In [ ]:
print(df_work2.dtypes.value_counts())

In [ ]:
pca_show(df_work2[df_work2["Hemetype"]=="Heme B"],num_comp =3,order = [1,2,0],best_par=3)

In [ ]:
pca_show(df_work2[df_work2["Hemetype"]=="Heme B"])

In [ ]:
pca_show(df_work2)

In [ ]:
print(p.components_[0].max(),  df_work2.columns[np.where(p.components_[0] == p.components_[0].max())[0][0] ]  )
print(p.components_[1].max(),df_work2.columns[ np.where(p.components_[1] == p.components_[1].max())[0][0]]   )
print(p.components_[2].max(), df_work2.columns[np.where(p.components_[2] == p.components_[2].max())[0][0] ]  )
print(p.components_[3].max(), df_work2.columns[np.where(p.components_[3] == p.components_[3].max())[0][0]  ] )
print(p.components_[4].max(),df_work2.columns[ np.where(p.components_[4] == p.components_[4].max())[0][0]  ] )
print(p.components_[5].max(), df_work2.columns[np.where(p.components_[5] == p.components_[5].max())[0][0]  ] )

In [ ]:
df_work2.columns[133]

In [ ]:
######df_calc

In [ ]:
_dihedral

Read in csv-files and change axes so that the PDB entries are a) non duplicate and b) the index column of the table so that we can clearly identify each heme by it's originating protein structure

In [ ]:
dfex_lit

In [ ]:
pd.read_csv("tables/Hemetypes.csv")

In [ ]:
#_4spin = pd.read_csv("4spin.csv")
_dihedral = pd.read_csv("tables/Dihedral.csv")
_nbo_singulet = pd.read_csv("nbo_singulet.csv")
_4spin = _4spin.drop("Unnamed: 0", axis=1)
_4spin = _4spin.set_index("PDB")
spin = _4spin
_dihedral = _dihedral.rename(columns={"index": "PDB"})
_dihedral["PDB"] = _dihedral["PDB"]#.str.upper()
_dihedral = _dihedral.set_index("PDB")
dihe = _dihedral
_nbo_singulet = _nbo_singulet.drop("Unnamed: 0", axis=1)
_nbo_singulet["PDB"] = _nbo_singulet["PDB"].str.upper()
_nbo_singulet = _nbo_singulet.set_index("PDB")
nbo = _nbo_singulet

Compare column names as a list to see which information is redundant, and can be used to potentially join the tables. Compare shapes of the resulting subtables with only the information from the inner join (columns that both tables contain) and add the lines that are omitted from the original tables because the calculations did compute succesfully in chloroform solution. 

In [ ]:
nbo_cols = list(nbo.columns)
dihe_cols = list(dihe.columns)
spin_cols = list(spin.columns)
insec1 = list(set(nbo_cols).intersection(spin_cols))
sub_nbo, sub_spin, i = pd.DataFrame(), pd.DataFrame(), 0
for col_name in insec1:
    sub_nbo.insert(loc=i, column=col_name, value=nbo[col_name])
    sub_spin.insert(loc=i, column=col_name, value=spin[col_name])
    i+=1
#print("sub_nbo shape: "+str(sub_nbo.shape))
#print("sub_spin shape: "+str(sub_spin.shape))
#here, add the lines that are missing, so both tables have the same format

In [ ]:
sub_nbo

Have a look at the distributions of the measured dihedral angles in the porphyrin ring

In [ ]:
#print("  \n\n\n\n\n ")
print("          {}---{}					".format("C3_NW","C3_NO"))
print("            /      \ ")
print("        {}      {}					".format("C1_NW","C1_NO"))
print("       /    \      /  \ ")
print("   {}        {}     {}					".format("WC2N","N_N","NC2O"))
print("     |          |        | 					")
print("  __{}       |        {}_					".format("C1_WN","C1_ON"))
print("{}    \      |      /      {}					".format("C3_WN","C3_ON"))
print("|        {}----{}----{}      |  					".format("NW","Fe","NO"))
print("{}__  /      |      \    _{}					".format("C3_WS","C3_OS"))
print("    {}       |       {}					".format("C1_WS","C1_OS"))
print("     |          |         | 					")
print("     {}      {}      {}					".format("SC2W","N_S","OC2S"))
print("       \      /   \    /  ")
print("        {}       {}					".format("C1_SW","C1_SO"))
print("            \     /")
print("          {}---{}						".format("C3_SW","C3_SO"))

In [ ]:
saddling = [["C1_SW", "N_S" ,"N_N","C1_NW"],["C1_SO", "N_S","N_N", "C1_NO"],["C1_WS", "NW", "NO" ,"C1_OS"],["C1_WN", "NW", "NO" ,"C1_ON"]]
ruffling = [["C3_SW", "C1_SW", "C1_WS", "C3_WS"],["C3_SO", "C1_SO", "C1_OS", "C3_OS"],["C3_WN", "C1_WN", "C1_NW", "C3_NW"],["C3_NO", "C1_NO", "C1_ON", "C3_ON"]]

In [ ]:
if not True or not True:
    print("i")

In [ ]:
dihe.head()
#dihe.max()
#dihe.min()
#dihe

In [ ]:
class angles():
    def __init__(self):
        return

    def transform_angles(angle):
        """
        angles.transform_angles transforms all angles lower than -30 by adding 360°.
        """
        if angle <= -30.0:
            angle=360+angle
        return angle

    def histplot_all(data):
        """ 
        angles.histplot_all() plots all columns of a pd.DataFrame containing e.g. dihedral angles into one plot, adds the kde. Beforehand, angles are transformed 
        with angles.transform_angles().
        """
        if not isinstance(data, pd.DataFrame):
            raise ValueError("data is not a pd.DataFrame.")
        dihe_trans = dihe.applymap(angles.transform_angles)
        fig = plt.figure()
        ax = fig.add_subplot()
        ax = sns.histplot(data=dihe_trans, binrange=[-30,210], bins=180, kde=True, alpha=0.4)
        ax.set_title(f"Dihedral angles")
        ax.set_xlabel(f"Angle [°]")
        fig.savefig(f"heme/histplot_dihe_all.png")
        return

    def histplot(data, column_name):
        """ 
        angles.histplot() plots a single column of a pd.DataFrame containing angles as a histogram. 
        """
        fig = plt.figure()
        ax = fig.add_subplot()
        ax = sns.histplot(data, binrange=[-30,210], bins=180, kde=True, alpha=0.4)
        ax.set_title(f"Dihedral angle {column_name}")
        ax.set_xlabel(f"{column_name} [°]")
        fig.savefig(f"heme/histplot_{column_name}.png")
        return

    def single_histplots(data):
        """ 
        angles.single_histplots() loops over all columns in the pd.DataFrame given, creating histograms with angles.histplot(). Beforehand, 
        angles are transformed with angles.transform_angles().
        """
        if not isinstance(data, pd.DataFrame):
            raise ValueError("data is not a pd.DataFrame.")
        data = data.applymap(angles.transform_angles)
        for column in data.columns:
            angles.histplot(data=data[column], column_name=column)
        return

    def histplot_grid(data):
        """ 
        angles.histplot_grid() takes a pd.DataFrame and creates a plot of all the columns, and arranges them in a grid. Beforehand, 
        angles are transformed with angles.transform_angles().
        """
        if not isinstance(data, pd.DataFrame):
            raise ValueError("data is not a pd.DataFrame.")
        data = data.applymap(angles.transform_angles)
        #nrows, ncols = (data.shape[1]//3)+1, 3
        #fig = plt.subplots(ncols=ncols, nrows=nrows)
        plt.figure(figsize=(15, 12))
        plt.subplots_adjust(hspace=0.7)
        plt.suptitle("Dihedral Angles of Heme-Porphyrin Ring in Protein Structures", fontsize=16, y=0.95)

        # loop through the length of tickers and keep track of index
        for n, column in enumerate(dihe.columns):
            # add a new subplot iteratively
            ax = plt.subplot(5, 3, n + 1)

            # filter df and plot ticker on the new subplot axis
            dihe[column].hist(ax=ax, bins=60)

            # chart formatting
            ax.set_title(column.upper())
            ax.set_xlim([-20, 220])
            #ax.get_legend().remove()
            ax.set_xlabel("Angle [°]")
            ax.set_ylabel("Count")
        plt.savefig(f"heme/histplot_grid.png")
        return

In [ ]:
#angles.histplot_all(data=dihe)
#angles.single_histplots(data=dihe)
angles.histplot_grid(data=dihe)

In [ ]:
#dist = pd.DataFrame(np.ones([37,14]))
#fig = plt.figure()
#ax = fig.add_subplot(projection="polar")
#c = ax.scatter(dihe.applymap(angles.transform_angles), dist)
#This makes no sense, why are all the data points shown somewhere they're not?